# 데이터 불러오기

In [ ]:
import pandas as pd

# 데이터 로드
data = pd.read_csv('your.csv', encoding='utf-8')

# 데이터 확인 (예시: 상위 5개 행 출력)
data.tail()

In [ ]:
# 데이터프레임에서 원문(사투리)과 번역(표준어) 열을 추출하여 리스트에 저장
Original_sentences = data['Original'].tolist()
translation_sentences = data['Translation'].tolist()

# 리스트에 저장된 문장 확인 (예시: 상위 5개 문장 출력)
print(Original_sentences[:5])
print(translation_sentences[:5])

In [ ]:
# 처음 10개 문장 쌍 출력
for Original, translation in zip(Original_sentences[:10], translation_sentences[:10]):
    print(f'[사투리]: {Original}')
    print(f'[표준어]: {translation}\n')


In [ ]:
# 사투리 토크나이저 훈련 데이터 제작
with open('train_Original.txt', 'w', encoding='utf-8') as f:
    for Original_sentence in Original_sentences:
        f.write(Original_sentence + '\n')

# 표준어 토크나이저 훈련 데이터 제작
with open('train_translation.txt', 'w', encoding='utf-8') as f:
    for translation_sentence in translation_sentences:
        f.write(translation_sentence + '\n')


## Okt로 모든 데이터 토큰화 진행
### 원문과 번역문 병렬 구조쌍 만들기 (+토큰화)

토큰화와 병렬 문장 쌍 생성의 순서는 중요합니다. 일반적으로 다음과 같은 순서로 작업을 진행합니다:

원문과 번역문 데이터 로드: 먼저 원문과 번역문 데이터를 로드합니다.

토큰화: 원문 및 번역문을 토큰화하여 각각의 토큰 목록을 생성합니다. 이때, 토큰화는 언어 모델 또는 형태소 분석기를 사용하여 수행할 수 있습니다.

병렬 문장 쌍 생성: 원문과 번역문의 토큰 목록을 이용하여 병렬 문장 쌍을 생성합니다. 쌍은 원문 문장과 해당 번역문 문장으로 구성됩니다.

순서를 지키지 않으면 토큰화를 하기 전에 데이터가 무작위로 섞일 수 있으며, 이로 인해 원문과 번역문 간의 대응 관계가 깨질 수 있습니다. 따라서 일반적으로 위의 순서를 따르는 것이 좋습니다.

In [ ]:
import csv
import json
import os
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 데이터 디렉토리 경로
base_directory = '/content/drive/MyDrive/jeju/'

# CSV 파일 경로 설정 (원문 및 번역 모두 저장)
csv_file = '/content/drive/MyDrive/jeju/parallel_data.csv'

# CSV 파일 헤더 설정
csv_columns = ['Utterance', 'Original', 'Translation']

# 데이터 저장 리스트
parallel_data = []

# 모든 데이터 폴더 선택 (Training 및 Validation)
data_directories = [
    os.path.join(base_directory, 'Training'),
    os.path.join(base_directory, 'Validation')
]

# 원문 및 번역 데이터를 읽어와서 병렬 구조 쌍으로 만들기
for data_directory in data_directories:
    data_files = [os.path.join(root, file) for root, dirs, files in os.walk(data_directory) for file in files if file.endswith('.json')]

    for data_file in data_files:
        with open(data_file, 'r', encoding='utf-8') as file:
            data_json = json.load(file)
            for idx, utterance in enumerate(data_json['utterance']):
                original_text = utterance['dialect_form']
                translation_text = utterance['standard_form']  # 번역 데이터로 사용할 열 선택

                # 토큰화 수행
                original_tokens = okt.morphs(original_text)
                translation_tokens = okt.morphs(translation_text)

                parallel_data.append({
                    'Utterance': f'Utterance_{idx + 1}',
                    'Original': ' '.join(original_tokens),  # 토큰을 공백으로 구분하여 저장
                    'Translation': ' '.join(translation_tokens)  # 토큰을 공백으로 구분하여 저장
                })

# CSV 파일에 병렬 데이터 저장
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()  # 헤더 쓰기

    for parallel_item in parallel_data:
        writer.writerow({
            'Utterance': parallel_item['Utterance'],
            'Original': parallel_item['Original'],
            'Translation': parallel_item['Translation']
        })

print(f"병렬 데이터가 {csv_file} 경로에 저장되었습니다.")

### 토큰을 추가하는 후처리 로직
각 언어 쌍의 평균 길이와 최대 길이를 계산하고,
[PAD] 토큰을 추가하여 모든 문장을 최대 길이로 맞추는 작업을 수행합니다.

### 모든 데이터셋을 텐서형 데이터로 변환
- 전처리와 후처리를 모두 마친 데이터들을 torch.Tensor로 변환해줍니다.
- 변환 후, DataLoader를 활용해 데이터들을 배치로 만들어줍니다.

In [ ]:
import csv
import torch
from torch.utils.data import DataLoader, TensorDataset
from konlpy.tag import Okt
from tqdm import tqdm

# Okt 형태소 분석기 초기화
okt = Okt()

# CSV 파일 경로 설정
csv_file = '/content/drive/MyDrive/jeju/parallel_data.csv'

# 데이터를 저장할 리스트
parallel_data = []

# CSV 파일에서 모든 데이터 읽어오기
with open(csv_file, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        parallel_data.append({
            'Utterance': row['Utterance'],
            'Original': row['Original'],
            'Translation': row['Translation']
        })

# 추가 후처리 작업 (SOS, EOS, PAD 토큰 추가)
max_len = 32  # 문장의 최대 길이를 지정합니다.
for item in parallel_data:
    item['Original'] = '[SOS] ' + item['Original'] + ' [EOS]'
    item['Translation'] = '[SOS] ' + item['Translation'] + ' [EOS]'

# 단어를 정수로 매핑하는 사전을 생성합니다.
word_to_index = {}
index_to_word = {}

# 특수 토큰을 사전에 추가합니다.
special_tokens = ['[PAD]', '[SOS]', '[EOS]']
for token in special_tokens:
    word_to_index[token] = len(word_to_index)
    index_to_word[len(word_to_index) - 1] = token

# 토큰화 및 후처리 함수
def tokenize_and_preprocess(sentence, max_len):
    tokens = okt.morphs(sentence)[:max_len-2]  # 최대 길이를 고려하여 자르기
    tokens = ['[SOS]'] + tokens + ['[EOS]']
    tokens += ['[PAD]'] * (max_len - len(tokens))
    return tokens

# 데이터를 토큰화 및 후처리하여 텐서로 변환하는 함수
def process_data(data, max_len):
    tokenized_data = [tokenize_and_preprocess(item['Original'], max_len) for item in tqdm(data, desc="Tokenizing Data")]  # tqdm 추가
    tensor_data = [torch.tensor(tokens_to_indices(tokens, max_len)).long() for tokens in tqdm(tokenized_data, desc="Converting to Tensors")]  # tqdm 추가
    return tensor_data

# 토큰을 정수 인덱스로 매핑하는 함수
def tokens_to_indices(tokens, max_len):
    indices = [word_to_index.get(token, word_to_index['[PAD]']) for token in tokens]
    indices += [word_to_index['[PAD]']] * (max_len - len(indices))
    return indices

# 문장의 최대 길이를 계산합니다.
jeju_lengths = [len(tokenize_and_preprocess(item['Original'], max_len)) for item in parallel_data]
std_lengths = [len(tokenize_and_preprocess(item['Translation'], max_len)) for item in parallel_data]

# 제주도 사투리 데이터의 평균 및 최대 길이 계산
average_jeju_length = sum(jeju_lengths) / len(jeju_lengths)
max_jeju_length = max(jeju_lengths)

# 표준어 데이터의 평균 및 최대 길이 계산
average_std_length = sum(std_lengths) / len(std_lengths)
max_std_length = max(std_lengths)

# 수정할 문장의 최대 길이를 선택
# 여기서는 각각의 데이터 세트에 대한 최대 길이 중 더 큰 값을 선택합니다.
max_len = max(max_jeju_length, max_std_length)

# 데이터를 토큰화 및 후처리하여 텐서로 변환
jeju_tensors = process_data(parallel_data, max_len)
std_tensors = process_data(parallel_data, max_len)

# DataLoader를 사용하여 데이터를 배치로 만듭니다.
batch_size = 32  # 배치 크기 조정 가능
jeju_loader = DataLoader(TensorDataset(*jeju_tensors), batch_size=batch_size, shuffle=True)
std_loader = DataLoader(TensorDataset(*std_tensors), batch_size=batch_size, shuffle=True)

# 데이터 사이즈 확인
jeju_data_size = len(jeju_loader.dataset)
std_data_size = len(std_loader.dataset)

print(f"제주도 사투리 데이터의 전체 크기: {jeju_data_size} 문장")
print(f"표준어 데이터의 전체 크기: {std_data_size} 문장")

# 배치 수 계산
jeju_batches = len(jeju_loader)
std_batches = len(std_loader)

print(f"제주도 사투리 데이터의 배치 수: {jeju_batches} 배치")
print(f"표준어 데이터의 배치 수: {std_batches} 배치")


In [ ]:
import csv

# 처리된 데이터 파일 경로
processed_data_file = '/content/drive/MyDrive/jeju/parallel_data.csv'

# 데이터를 저장할 리스트
processed_data = []

# CSV 파일에서 데이터 읽어오기
with open(processed_data_file, 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        processed_data.append({
            'Utterance': row['Utterance'],
            'Original': row['Original'],
            'Translation': row['Translation']
        })

# 후처리 결과 확인을 위한 예시 문장 출력
num_examples = 5  # 출력할 예시 문장 개수
for i, item in enumerate(processed_data[:num_examples]):
    print(f"Example {i + 1}:")
    print(f"Original: {item['Original']}")
    print(f"Translation: {item['Translation']}")
    print("-" * 30)

In [ ]:
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 스페셜 토큰 정의
special_tokens = ['[PAD]', '[SOS]', '[EOS]']

# 스페셜 토큰에 대한 인덱스 매핑 딕셔너리 생성
special_token_to_index = {token: idx for idx, token in enumerate(special_tokens)}

# 스페셜 토큰에 대한 인덱스를 얻을 수 있습니다.
pad_idx = special_token_to_index.get('[PAD]')
sos_idx = special_token_to_index.get('[SOS]')
eos_idx = special_token_to_index.get('[EOS]')

In [ ]:
# 예시 문장 (처리한 데이터 중 하나를 선택)
sample_translation = parallel_data[51055]['Translation']

# 토큰화
sample_tokens = sample_translation.split()

# 후처리 작업 결과 (SOS, EOS, PAD 토큰 제외)
post_proc_sent = [word_to_index.get(token, word_to_index['[PAD]']) for token in sample_tokens if token not in ['[SOS]', '[EOS]', '[PAD]']]

print(f'후처리 결과: {post_proc_sent}\n')
print(f'후처리 해석: {" ".join(sample_tokens)}')

In [ ]:
dataset_size = len(parallel_data)
print(f"데이터셋의 크기: {dataset_size}")

# 데이터셋 크기가 283,929인 경우, 이 데이터셋에는 총 283,929개의 데이터 포인트가 있으며,
# 각 데이터 포인트에는 여러 개의 토큰이 아닌 원시 텍스트 형태로 문장 또는 문장 쌍이 포함되어 있을 것입니다.
# 따라서 토큰 수를 세려면 각 데이터 포인트를 토큰화한 다음 모든 토큰의 수를 합산한 것임.